In [1]:
from __future__ import absolute_import, print_function
import time
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torchvision.transforms as T
from torchvision.datasets import ImageFolder
from torchvision.models import resnet18, ResNet18_Weights, resnet34
from torch.utils.data import DataLoader
from tqdm import tqdm
from PIL import Image

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

c:\Users\Noel\anaconda3\envs\torchCUDA\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda:0


In [2]:
weights = None  # ResNet18_Weights.DEFAULT
model = resnet34(weights=weights)

# reshape last layer.
in_features = model.fc.in_features
model.fc = nn.Linear(in_features, 10)
model.load_state_dict(torch.load(r"C:\Users\Noel\Documents\THESIS\Feature Visualization\Weights\resnet34_torchvision\test72_epoch446.pth"))
# Set model to evaluation mode and send to device
model.to(device).eval()

layers_of_interest = [name for name, _ in model.named_modules() if "conv" in name or "fc" in name]

print(layers_of_interest)


['conv1', 'layer1.0.conv1', 'layer1.0.conv2', 'layer1.1.conv1', 'layer1.1.conv2', 'layer1.2.conv1', 'layer1.2.conv2', 'layer2.0.conv1', 'layer2.0.conv2', 'layer2.1.conv1', 'layer2.1.conv2', 'layer2.2.conv1', 'layer2.2.conv2', 'layer2.3.conv1', 'layer2.3.conv2', 'layer3.0.conv1', 'layer3.0.conv2', 'layer3.1.conv1', 'layer3.1.conv2', 'layer3.2.conv1', 'layer3.2.conv2', 'layer3.3.conv1', 'layer3.3.conv2', 'layer3.4.conv1', 'layer3.4.conv2', 'layer3.5.conv1', 'layer3.5.conv2', 'layer4.0.conv1', 'layer4.0.conv2', 'layer4.1.conv1', 'layer4.1.conv2', 'layer4.2.conv1', 'layer4.2.conv2', 'fc']


This cell works, but is technically wrong and would not 'fly' if the notebook was to be converted to an actual python script.

In [3]:
# layer_activations = {}

# def hook_wrapper(name: str):
#     def hook_fn(module: nn.Module, input: torch.Tensor, output: torch.Tensor) -> None:
#         layer_activations[name] = output
#     return hook_fn


# for name, layer in model.named_modules():
#     if name in layers_of_interest:
#         layer.register_forward_hook(hook_wrapper(name))

To combat this, I made the hooks into objects of a hook class, holding both the output and the hook function, thus creating dictionary entries 
of a key/value pair of name/Hook_Layer object.

In [3]:
class Hook_Layer():
    def __init__(self, layer) -> None:
        self.hook = layer.register_forward_hook(self.hook_fn)
        self.output = None

    def hook_fn(self, layer, input, output):
        self.output = output
    
    def __call__(self):
        return self.output

layer_activations = {}
for name, layer in model.named_modules():
    if isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear):
        layer_activations[name] = Hook_Layer(layer)

print(layer_activations.values())

dict_values([<__main__.Hook_Layer object at 0x000002DB35D05120>, <__main__.Hook_Layer object at 0x000002DB35D06EF0>, <__main__.Hook_Layer object at 0x000002DB35D064A0>, <__main__.Hook_Layer object at 0x000002DB35D06320>, <__main__.Hook_Layer object at 0x000002DB35D06230>, <__main__.Hook_Layer object at 0x000002DB35D060E0>, <__main__.Hook_Layer object at 0x000002DB35D05F60>, <__main__.Hook_Layer object at 0x000002DB35D05690>, <__main__.Hook_Layer object at 0x000002DB35D05630>, <__main__.Hook_Layer object at 0x000002DB35D06590>, <__main__.Hook_Layer object at 0x000002DB35D066B0>, <__main__.Hook_Layer object at 0x000002DB35D06830>, <__main__.Hook_Layer object at 0x000002DB35D06950>, <__main__.Hook_Layer object at 0x000002DB35D06A70>, <__main__.Hook_Layer object at 0x000002DB35D06BF0>, <__main__.Hook_Layer object at 0x000002DB35D06D10>, <__main__.Hook_Layer object at 0x000002DB35D07A30>, <__main__.Hook_Layer object at 0x000002DB35D07C70>, <__main__.Hook_Layer object at 0x000002DB35D07D30>,

In [4]:
# Create a dataset class that extends ImageFolder while
# simultaneously returning a 3 way Tuple, instead of the
# original that contains 2 elements.
# For that reason we must define a new __getitem__ method.
class ImageFolderWithPaths(ImageFolder):
    """Dataset class extending ImageFolder dataset,
        returning Tuple.
        
        Returns:
                Tuple[img[torch.Tensor],
                      label[int],
                      path[str]]
        """
    def __getitem__(self, index: int):
        # Super the __getitem__ of base class
        img, label = super().__getitem__(index)
        # Extract the path of each image in the dataset
        path = self.imgs[index][0]
        # Return new tuple with 3 elements
        return (img, label, path)

    

In [5]:
batch_size = 64


transforms = T.Compose([T.Resize(224),
                        # T.CenterCrop(224),
                        T.ToTensor(),
                        T.Normalize([0.5162, 0.4644, 0.3975],
                                    [0.2724, 0.2640, 0.2574])
                        ])

dataset = ImageFolderWithPaths(root=r"C:\Users\Noel\Documents\THESIS\Outputs_Feature_Visualization\test72outputs",
                               transform=transforms)

dataloader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=False, num_workers=0)
print("Dataloader Initialized. Note that workers > 1 cannot be specified when not in main.")
# ================================================================
data = [] # list of dicts to be filled with dicts...

with torch.no_grad():
    for images, label, paths in tqdm(dataloader, total=len(dataloader)):
        # Send stuff to GPU if available.
        images = images.to(device)
        label = label.to(device)
        # Make Forward Pass.
        outputs = model(images)
        _, preds = torch.max(outputs, dim=1)
        # path_list = []
        # for path in paths:
        #     path_list.append(path)
        
        for i, image in enumerate(images):
            private_dict = {}
            # Three entries regarding the image identification.
            # private_dict['path'] = path_list[i]
            private_dict['path'] = paths[i]
            private_dict['layer'] = label[i].item()
            private_dict['prediction'] = preds[i].item()
            # Iterate over all available layers.
            for key, hook_object in layer_activations.items():
                tensor_out = hook_object()  # .output  # modified from original script to accommodate objects 
                if key == 'fc':
                    # The array to store is a 32 by 10 array, each batch
                    output = torch.unbind(tensor_out, dim=0)
                else:
                    # The array will have a final shape of 32 by num_channels
                    # in specific layer
                    b, c, _, _ = tensor_out.shape
                    output = torch.unbind(tensor_out.view(b, c, -1).mean(2), dim=0)
                private_dict[key] = output[i].cpu().numpy()
            data.append(private_dict)

Dataloader Initialized. Note that workers > 1 cannot be specified when not in main.


100%|██████████| 262/262 [06:44<00:00,  1.54s/it]


In [6]:
df = pd.DataFrame(data, copy=False)
# df.head()

In [7]:
df.to_parquet(r"C:\Users\Noel\Documents\THESIS\Feature Visualization\Dataframes\reapeatability_resnet34.parquet")

In [8]:
df = pd.read_parquet(r"C:\Users\Noel\Documents\THESIS\Feature Visualization\Dataframes\reapeatability_resnet34.parquet")
df.tail()

,path,layer,prediction,conv1,layer1.0.conv1,layer1.0.conv2,layer1.1.conv1,layer1.1.conv2,layer1.2.conv1,layer1.2.conv2,...,layer3.5.conv1,layer3.5.conv2,layer4.0.conv1,layer4.0.conv2,layer4.0.downsample.0,layer4.1.conv1,layer4.1.conv2,layer4.2.conv1,layer4.2.conv2,fc
0,C:\Users\Noel\Documents\THESIS\Outputs_Feature...,0,8,"[-0.0033745547, -0.42918676, 0.18082474, 0.006...","[-0.034679383, -0.038424972, 0.010864167, -0.0...","[-0.0020865758, 0.008425573, 0.0040470865, -0....","[0.0053859754, -0.007726112, 0.0013055743, -0....","[-0.0027077484, 0.0011912129, -0.0010511606, -...","[-0.0005719438, 0.0024042143, -0.0024090451, -...","[-0.0041987267, 5.974373e-06, -0.0005231125, 0...",...,"[-0.006232737, -0.005764516, -0.009423526, -0....","[2.9387807e-05, -0.0003003147, -0.0018214167, ...","[-0.001937044, -0.0024104144, -0.0020212522, -...","[-0.0024027778, -0.0025440508, -0.002051234, 0...","[-0.0021192597, -0.0026732688, -0.00086916116,...","[-0.0013890853, 0.007361405, -0.019993545, 0.0...","[-0.0026025353, -0.0030683135, -0.0017848288, ...","[-0.021023093, 0.011987248, -0.009938622, 0.00...","[-0.0035901996, -0.0027782156, -0.0026272011, ...","[0.39717865, -0.50606644, 0.37344494, -0.92132..."
1,C:\Users\Noel\Documents\THESIS\Outputs_Feature...,0,8,"[0.004638297, 0.50511456, -0.17607494, -0.0067...","[-0.08193014, -0.00519906, 0.025840944, 0.0283...","[-0.0061499607, -0.012112749, 0.0025921755, -0...","[0.0066752615, -0.005129631, 0.0076790364, -0....","[0.0019916294, 0.0034686392, 0.0012062425, 0.0...","[-0.0060652383, 0.0034615411, -0.0052531837, 0...","[-0.0029019644, -0.00029531628, -4.11923e-06, ...",...,"[-0.005052153, -0.0061707, -0.005824093, -0.00...","[5.861327e-05, -0.00044124128, -0.001199095, -...","[0.0010404609, -0.0010269147, -0.0014222037, -...","[-0.0021817256, -0.0026621902, -0.0007249894, ...","[-0.0017001407, -0.0022356424, -0.00048238251,...","[-0.0034134204, 6.723345e-05, -0.014172575, 0....","[-0.0031539383, -0.003767936, -0.0007053565, 0...","[-0.007292089, 0.005875447, -0.0037730103, 0.0...","[-0.0048597185, -0.0036083423, -0.0020547514, ...","[1.3594136, -1.1047592, 0.49673295, -0.73394, ..."
2,C:\Users\Noel\Documents\THESIS\Outputs_Feature...,0,4,"[0.0025449453, 0.45937902, -0.22739105, -0.007...","[-0.0822344, -0.008479998, 0.022766367, 0.0345...","[-0.009853885, -0.012154875, 0.00045066574, -0...","[0.0021497642, -0.0070615956, 0.0048149345, -0...","[-0.00021139231, 0.0029254372, 0.0013999082, 0...","[-0.00443534, -1.3336206e-05, -0.0035291645, -...","[-0.0042086425, -0.0014527297, -0.0002653038, ...",...,"[-0.0035002413, -0.0049959756, -0.002733205, -...","[0.00024305728, 0.00044383868, -0.0011485392, ...","[0.0054426994, -0.00039021784, -0.0009553507, ...","[0.0010423608, 0.005437585, 0.0030866794, 0.00...","[-0.00014272399, 0.0018853407, 0.00062146306, ...","[-0.0029565417, -0.009451935, 0.00068953953, -...","[0.00037604768, 0.00600729, 0.0037275741, -0.0...","[0.0022630785, -0.04643796, -0.007821188, -0.0...","[-0.00042004365, 0.0070401076, 0.0031331284, 0...","[1.1662073, -1.4986442, 0.821904, 1.5146831, 2..."
3,C:\Users\Noel\Documents\THESIS\Outputs_Feature...,0,8,"[-0.0012989846, -0.38239875, 0.22895174, 0.007...","[-0.04401641, -0.042307444, 0.0154071655, -0.0...","[-0.0065884953, 0.00258441, 0.006492888, -0.00...","[0.0033897555, -0.0056121764, 0.0061996323, 0....","[-0.00042871878, -0.00085110305, -0.0004634733...","[-0.005519734, -0.0008250423, -0.001520291, -0...","[-0.0052961945, 5.8215563e-05, -0.0011764986, ...",...,"[-0.0041921376, -0.007768304, -0.006750722, -0...","[8.325061e-05, 0.0005475663, -0.0027501553, -0...","[0.0003085709, -0.004177123, -0.0035730058, -0...","[-0.00061419007, 0.0032424396, 0.00024514613, ...","[-0.0013108528, 0.00016468721, -0.00012460812,...","[0.017739113, 0.0033298992, -0.017306287, 0.00...","[-0.00048541217, 0.0042410516, 0.0005480753, 0...","[-0.021662625, 0.00018623882, 0.011916201, 0.0...","[-0.0010173022, 0.005967915, -0.00024580248, 0...","[0.9556